In this notebook, we will calculate features numerically (without trotterization).

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# GPU check
!nvidia-smi -L

zsh:1: command not found: nvidia-smi


In [5]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys

# プロジェクトのルートディレクトリに移動
project_root = os.path.dirname(os.getcwd())
os.chdir(project_root)

# パッケージのルートディレクトリをパスに追加
sys.path.append(os.path.join(project_root, "fourier_learning_ibm"))

from fourier_learning_ibm.heisenberg import (
    HeisenbergModel,
    HeisenbergModelGPU,
    get_graph,
)
from fourier_learning_ibm.setup import setup_backend
from fourier_learning_ibm.helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math
import time

ModuleNotFoundError: No module named 'cupy'

In [4]:
import warnings

warnings.filterwarnings("ignore")

### Config backend

In [5]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['reset', 'sx', 'cz', 'measure', 'x', 'id']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('reset', (0,)), ('reset', (1,)), ('r

### Prepare parameters and load dataset

In [6]:
n_qubits = 12
C = 3
# C = 3 * (n_qubits - 1)
threshold = 0.6

# Load dataset (orient="records")
df_n_step = pd.read_json(
    f"results/get_param/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
    orient="records",
)
n_samples = 55

# Note that K = n_features-1.
# n_features counts only the number of real features, also the one of t=0, i.e., constant term.
# The number of all features, including imaginary numbers, is 2K+1.
n_features = len(df_n_step)

times = df_n_step["t"].values

display(df_n_step)
print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_features: {n_features}")
print(f"times: {times}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json", orient="records"
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

,t,n_step,fidelity
0,0.000000,1,1.000000
1,1.047198,1,0.876665
2,2.094395,1,0.874939
3,3.141593,1,0.865199
4,4.188790,1,0.830213
5,5.235988,2,0.770913
6,6.283185,2,0.762874
7,7.330383,2,0.744273
8,8.377580,2,0.706917
9,9.424778,3,0.670175


n_qubits: 12
C: 3
threshold: 0.6
n_samples: 55
n_features: 12
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.0954419343, -0.0212937804, 0.1249298174, 0....",0.792109,0.792109,9.820000e-08
1,1,"[0.1656462529, 0.0558403774, 0.1252779708, -0....",1.102887,1.102887,1.869000e-07
2,2,"[0.1404127689, 0.11727099390000001, 0.08304389...",1.130663,1.130663,1.148000e-07
3,3,"[-0.061944959, -0.014558126800000001, -0.14849...",1.552866,1.552865,3.841000e-07
4,4,"[0.11249198690000001, 0.1031829549, -0.0380960...",1.149036,1.149036,2.388000e-07
5,5,"[0.1515760814, -0.0221829062, 0.0371159373, -0...",1.332599,1.332599,2.610000e-08
6,6,"[0.022335651800000002, -0.07395899900000001, -...",1.114135,1.114134,4.651000e-07
7,7,"[0.0633589424, 0.022317037, 0.1110911404000000...",1.034106,1.034106,8.054000e-07
8,8,"[-0.1461354535, 0.00045074100000000005, -0.150...",1.421492,1.421492,1.777000e-07
9,9,"[0.1377260837, 0.1218283242, 0.0595475655, -0....",0.634733,0.634733,5.380000e-08


# Calculate Fourier feature

## Exact (No Trotterization)
Exact means that we compute the matrix exponential of the Hamiltonian, not Trotterized.

In [ ]:
data = []
probs_phase0_exact = {}
probs_phase1_exact = {}
probs_phase2_exact = {}
probs_phase3_exact = {}

# Initialize the save file
with open(f"results/fourier_feature_exact/temp_progress.txt", "w") as f:
    f.write("")  # clear the file

for i in range(n_samples):
    # Show progress and save file
    progress_report = f"Calculating features for sample {i}/{n_samples}"
    print(progress_report)
    with open(f"results/fourier_feature_exact/temp_progress.txt", "a") as f:
        f.write(progress_report + "\n")

    Js = all_Js[i]
    G = get_graph(n_qubits, Js)
    heisenberg = HeisenbergModelGPU(n_qubits, G)

    lambda_ref = np.sum(Js)  # Reference eigenvalue.
    features_exact = []

    probs_phase0_exact[f"sample{i}"] = {}
    probs_phase1_exact[f"sample{i}"] = {}
    probs_phase2_exact[f"sample{i}"] = {}
    probs_phase3_exact[f"sample{i}"] = {}

    # Compute the Fourier features for different times
    for k in range(n_features):
        start = time.time()
        final_state_phase0 = heisenberg.exact_simulation(times[k], phase=0)
        final_state_phase1 = heisenberg.exact_simulation(times[k], phase=1)
        final_state_phase2 = heisenberg.exact_simulation(times[k], phase=2)
        final_state_phase3 = heisenberg.exact_simulation(times[k], phase=3)

        # If “00...00” is not present, the probability is 0.
        prob_phase0 = final_state_phase0.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase1 = final_state_phase1.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase2 = final_state_phase2.probabilities_dict().get("0" * n_qubits, 0)
        prob_phase3 = final_state_phase3.probabilities_dict().get("0" * n_qubits, 0)

        probs_phase0_exact[f"sample{i}"][f"f_{k}"] = prob_phase0
        probs_phase1_exact[f"sample{i}"][f"f_{k}"] = prob_phase1
        probs_phase2_exact[f"sample{i}"][f"f_{k}"] = prob_phase2
        probs_phase3_exact[f"sample{i}"][f"f_{k}"] = prob_phase3

        inner_product = np.exp(-1j * lambda_ref * times[k]) * (
            (prob_phase0 - prob_phase1) + 1j * (prob_phase2 - prob_phase3)
        )

        features_exact.append(inner_product.real)
        if k != 0:
            features_exact.append(inner_product.imag)
        end = time.time()
        elapsed_time = end - start

        # Show progress and save file
        progress_report = (
            f"Time: {times[k]:.3f}, Feature: {k}, Elapsed time: {elapsed_time:.2f}[s]"
        )
        print(progress_report)
        with open(f"results/fourier_feature_exact/temp_progress.txt", "a") as f:
            f.write(progress_report + "\n")
    data.append([i, *features_exact, all_expected_values[i]])
    print()

# Create column names for the DataFrame
columns = []
columns.append("sample_id")
for k in range(n_features):
    columns.append(f"f_{k} Re")
    if k != 0:
        columns.append(f"f_{k} Im")
columns.append("expected_value")

# Convert to a DataFrame
df_exact = pd.DataFrame(data, columns=columns)
display(df_exact)

# Save the features
path = f"results/fourier_feature_exact/{n_qubits}Q"
df_exact.to_json(f"{path}/features.json", orient="records", indent=4)

# Save the probabilities, for reference
with open(f"{path}/probs_phase0.json", "w") as f:
    json.dump(probs_phase0_exact, f)

with open(f"{path}/probs_phase1.json", "w") as f:
    json.dump(probs_phase1_exact, f)

with open(f"{path}/probs_phase2.json", "w") as f:
    json.dump(probs_phase2_exact, f)

with open(f"{path}/probs_phase3.json", "w") as f:
    json.dump(probs_phase3_exact, f)

Calculating features for sample 0/55
Time: 0.000, Feature: 0, Elapsed time: 16.11[s]
Time: 1.047, Feature: 1, Elapsed time: 14.89[s]
Time: 2.094, Feature: 2, Elapsed time: 14.97[s]
Time: 3.142, Feature: 3, Elapsed time: 15.05[s]
Time: 4.189, Feature: 4, Elapsed time: 15.01[s]
Time: 5.236, Feature: 5, Elapsed time: 15.35[s]
Time: 6.283, Feature: 6, Elapsed time: 15.05[s]
Time: 7.330, Feature: 7, Elapsed time: 15.25[s]
Time: 8.378, Feature: 8, Elapsed time: 15.18[s]
Time: 9.425, Feature: 9, Elapsed time: 15.15[s]
Time: 10.472, Feature: 10, Elapsed time: 15.31[s]
Time: 11.519, Feature: 11, Elapsed time: 15.33[s]

Calculating features for sample 1/55
Time: 0.000, Feature: 0, Elapsed time: 14.92[s]
Time: 1.047, Feature: 1, Elapsed time: 15.00[s]
Time: 2.094, Feature: 2, Elapsed time: 14.67[s]
Time: 3.142, Feature: 3, Elapsed time: 15.27[s]
Time: 4.189, Feature: 4, Elapsed time: 15.01[s]
Time: 5.236, Feature: 5, Elapsed time: 15.10[s]
Time: 6.283, Feature: 6, Elapsed time: 15.13[s]
Time: 7.3